In [1]:
!pip install -qU \
  pinecone-client[grpc]==2.2.1 \
  datasets==2.12.0 \
  sentence-transformers==2.2.2
     


zsh:1: no matches found: pinecone-client[grpc]==2.2.1


In [2]:
-Xfrozen_modules=off

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3597591255.py, line 1)

In [3]:
!pip3 install "pinecone-client[grpc]"

  Obtaining dependency information for grpcio>=1.44.0 from https://files.pythonhosted.org/packages/bb/1c/4741e490b93488e077e36543b0acbfece41314b5a1ffae05bc7e2e9b3375/grpcio-1.59.0-cp311-cp311-macosx_10_10_universal2.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 5.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-datasets 0.3.1a0 requires pandas<2.0.0,>=1.5.3, but you have pandas 2.1.1 which is incompatible.
pinecone-datasets 0.3.1a0 requires protobuf<3.20.0,>=3.19.3, but you have protobuf 3.20.3 which is incompatible.


SyntaxError: invalid syntax (709945722.py, line 1)

In [5]:
from datasets import load_dataset

dataset = load_dataset('quora', split='train[240000:320000]')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

Dataset quora downloaded and prepared to /Users/hammed/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04. Subsequent calls will reuse this data.


Dataset({
    features: ['questions', 'is_duplicate'],
    num_rows: 80000
})

In [6]:
questions = []

for record in dataset['questions']:
    questions.extend(record['text'])
  
# remove duplicates
questions = list(set(questions))
print('\n'.join(questions[:5]))
print(len(questions))
     

What would happen if you took a PS4 hardrive with OS installed and put it in a working PC?
Why is the Chinese language so important?
What is the best coaching center for GRE in Hyderabad?
Why do some people love to sleep so much?
What is the cut off for cse iit hyderabad for the year 2017?
136057


In [7]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [8]:
query = 'which city is the most populated in the world?'

xq = model.encode(query)
xq.shape

(384,)

In [9]:
_id = '0'
metadata = {'text': query}

vectors = [(_id, xq, metadata)]

In [10]:
import os
import pinecone

pinecone.init(
    api_key='093171bb-e130-491e-a6ca-5d9888ff0770',
    environment='gcp-starter'
)

In [11]:
index_name = 'semantic-search'

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine'
    )

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'Content-Length': '140', 'date': 'Tue, 17 Oct 2023 23:57:37 GMT', 'x-envoy-upstream-service-time': '31', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Capacity Reached. Starter Projects support a single index. Create a new project to add more. Your Starter Project remains free post-upgrade.


In [12]:
index_name = 'semantic-search'

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine'
    )

In [13]:
index = pinecone.GRPCIndex(index_name)

In [14]:
from tqdm.auto import tqdm

batch_size = 128

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

  0%|          | 0/1063 [00:00<?, ?it/s]

PineconeException: UNKNOWN:Error received from peer  {created_time:"2023-10-17T20:27:51.909-04:00", grpc_status:9, grpc_message:"Starter index record limit reached. Current number: 100419, records in request: 128, limit: 100000"}

In [15]:
query = "which city has the highest population in the world?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
xc

{'matches': [{'id': '49414',
              'metadata': {'text': 'What country has the biggest population?'},
              'score': 0.7657305,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '97181',
              'metadata': {'text': 'What is the biggest city?'},
              'score': 0.72731686,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '2208',
              'metadata': {'text': 'What is the most isolated city in the '
                                   'world, with over a million metro area '
                                   'inhabitants?'},
              'score': 0.70319796,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '17160',
              'metadata': {'text': 'Which is the most beautiful city in '
                                   'world?'},
              'score': 0.7003917,
     

In [16]:

for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.77: What country has the biggest population?
0.73: What is the biggest city?
0.7: What is the most isolated city in the world, with over a million metro area inhabitants?
0.7: Which is the most beautiful city in world?
0.69: What is the most beautiful city in the world?


In [17]:
query = "which metropolis has the highest number of people?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.67: What is the most isolated city in the world, with over a million metro area inhabitants?
0.64: What is the biggest city?
0.6: What is the most dangerous city in USA?
0.59: What country has the biggest population?
0.58: What are the most dangerous cities in the world?


In [18]:
query = "When did WWII end?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.88: When did world war 2 end?
0.82: When 2nd world war came to end?
0.77: How and when did WWI start and end?
0.73: What year did World War I start?
0.73: When did World War One start?


In [19]:
query = "Who was the first president of the US?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.98: Who was the first U.S. President?
0.77: Who is the U.S. President?
0.72: Who was the all time best President of the United States of America?
0.68: How many United States Presidents have there been?
0.68: Who's the best US president ever?


In [20]:
query = "What is the highest mountain in the world?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.99: Which is the highest mountain in the world?
0.86: What is the highest mountain in Europe?
0.86: What is the biggest mountain on earth?
0.85: What is the highest mountain in India?
0.84: Which is the highest peak of the world?
